<a href="https://colab.research.google.com/github/vmjs1234/TIMEGPT-TABULA-RDL/blob/main/Tabular/zero_shot_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-Shot Inference with Tabula Model
In this notebook, we demonstrate how to perform zero-shot inference using a Tabula model.

In [3]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the Tabula model and tokenizer
model_name = 'mlfoundations/tabula-8b'  # Replace with actual model name if different
# Use AutoModelForCausalLM for Llama-based models like Tabula
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [4]:
# Prepare tabular data for inference
# Example row of tabular data
row = {'age': 35, 'income': 50000, 'education': "Bachelor's"}  # Replace with your data

# Convert the row into a natural language prompt
prompt = f"The person's age is {row['age']}, their income is {row['income']}, and their education level is {row['education']}."

# Display the generated prompt
print(f'Generated Prompt: {prompt}')


Generated Prompt: The person's age is 35, their income is 50000, and their education level is Bachelor's.


In [7]:
from transformers import GenerationConfig

In [13]:
import signal
import os

class TimeoutException(Exception):
    pass


def timeout_handler(signum, frame):
    raise TimeoutException("Inference timed out")


# Perform zero-shot inference
def run_inference(prompt, timeout_seconds=300):  # Set timeout to 5 minutes (300 seconds)
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt')

    # Define generation configuration for faster inference
    generation_config = GenerationConfig(
        max_new_tokens=50,  # Limit the number of generated tokens
        num_beams=1,  # Use greedy decoding for faster inference
        do_sample=False,
    )

    # Set the timeout handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)

    try:
        # Generate predictions (inference) with generation configuration
        outputs = model.generate(**inputs, generation_config=generation_config)

        # Decode the output to get the prediction
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Display the prediction
        print(f'Prediction: {prediction}')

    except TimeoutException:
        print("Inference timed out. Returning partial results if available.")
        # You can add logic here to handle partial results if possible

    finally:
        signal.alarm(0)  # Disable the alarm

In [15]:
prompt = "what is the person's age ?"  # Replace with your actual prompt
run_inference(prompt)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Inference timed out. Returning partial results if available.


In [16]:
run_inference(prompt)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Inference timed out. Returning partial results if available.
